In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import re
from unidecode import unidecode
tqdm.pandas()

In [2]:
page = requests.get("https://thefangirlverdict.com/index/all-reviews/full-list-of-shows/")
soup = BeautifulSoup(page.content)

In [3]:
for p in soup.find_all("p", style="text-align:justify"):
    p.decompose()

In [4]:
names = []
urls = []
grades = []

for x in tqdm(range(700)):
    
    element = soup.select_one("#post-7658 > div > p:nth-child(" + str(x) + ")")
    if element is None:
        continue
    
    kdrama = element.findChildren("a" , recursive=False)
    #print(kdrama)
    grade = element.findChildren("strong", recursive=False)
    #print(grade)
    
    if kdrama:
        names.append(kdrama[0].contents[0])
        urls.append(kdrama[0].get("href"))
        if grade:
            grades.append(grade[0].text)
        else:
            grades.append("NA")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 700/700 [02:30<00:00,  4.64it/s]


In [5]:
df = pd.DataFrame({"source": names, "url": urls, "rating": grades})

In [6]:
df['rating'] = df['rating'].replace(['A++', 'A+', 'A', 'A-','B++','B+','B','B-','C++', 'C+', 'C', 'C-','D++','D+','D','D-'],['A1', 'A2', 'A3', 'A4','B1','B2','B3','B4','C1', 'C2', 'C3', 'C4','D1','D2','D3','D4'])

In [7]:
df.sort_values('rating', inplace=True)

In [8]:
df["year"] = df.apply(lambda row: re.search(r'\d+', row["url"]).group(), axis=1)
#df["source"] = df.apply(lambda row: unidecode(row["source"]), axis=1)
df.head(20)

,source,url,rating,year
437,The Slave Hunters [Chuno],http://thefangirlverdict.com/2013/07/27/review...,A1,2013
343,Secret Affair [Secret Love Affair],http://thefangirlverdict.com/2014/09/12/review...,A1,2014
62,Chuno [Slave Hunter],http://thefangirlverdict.com/2013/07/27/review...,A1,2013
280,Nirvana In Fire [Chinese Drama],http://thefangirlverdict.com/2015/12/09/flash-...,A1,2015
365,Slave Hunter [Chuno],http://thefangirlverdict.com/2013/07/27/review...,A1,2013
352,Secret Love Affair [Secret Affair],http://thefangirlverdict.com/2014/09/12/review...,A1,2014
230,Miss Granny [Movie],http://thefangirlverdict.com/2014/12/26/flash-...,A2,2014
504,Your Name [Japanese Movie],http://thefangirlverdict.com/2017/02/08/flash-...,A2,2017
317,Queen In-hyunâ€™s Man,http://thefangirlverdict.com/2012/12/12/review...,A2,2012
274,Navillera,https://thefangirlverdict.com/2021/06/06/revie...,A2,2021


In [ ]:
df.to_csv("df.csv", encoding='utf-8', index=False)